# Session 2b: Monte Carlo Methods in Grid World

## Colab Setup

In [1]:
#uncomment only if you're running from google colab
# !git clone https://github.com/Datatouille/rl-workshop
# !mv rl-workshop/* .
# !ls

## Imports

In [2]:
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
#cross check with our solutions once you finish
# from solutions.agents import GridworldAgent
from solutions.environments import Gridworld

## Fill in The Code

In [3]:
import numpy as np
from collections import defaultdict
import sys

"""
Coding assignment order:
1. get_v
2. get_q
3. mc_control_q
4. mc_control_glie
"""

class GridworldAgent:
    def __init__(self, env, policy, gamma = 0.9, 
                 start_epsilon = 0.9, end_epsilon = 0.1, epsilon_decay = 0.9):
        self.env = env
        self.n_action = len(self.env.action_space)
        self.policy = policy
        self.gamma = gamma
        self.v = dict.fromkeys(self.env.state_space,0)
        self.n_v = dict.fromkeys(self.env.state_space,0)
        self.q = defaultdict(lambda: np.zeros(self.n_action))
        self.n_q = defaultdict(lambda: np.zeros(self.n_action))
        self.start_epsilon = start_epsilon
        self.end_epsilon = end_epsilon
        self.epsilon_decay = epsilon_decay
    def get_epsilon(self,n_episode):
        epsilon = max(self.start_epsilon * (self.epsilon_decay**n_episode),self.end_epsilon)
        return(epsilon)
    def get_v(self,start_state,epsilon = 0.):
        episode = self.run_episode(start_state,epsilon)
        """
        Write the code to calculate the state value function of a state 
        given a deterministic policy.
        """
        v=0
        
        return(v)
    def get_q(self, start_state, first_action, epsilon=0.):
        episode = self.run_episode(start_state,epsilon,first_action)
        """
        Write the code to calculate the action function of a state 
        given a deterministic policy.
        """
        q=0
        return(q)
    def select_action(self,state,epsilon):
        probs = np.ones(self.n_action) * (epsilon / self.n_action)
        best_action = self.policy[state]
        probs[best_action] = 1 - epsilon + (epsilon / self.n_action)
        action = np.random.choice(np.arange(self.n_action),p=probs)
        return(action)
    def print_policy(self):
        for i in range(self.env.sz[0]):
            print('\n----------')
            for j in range(self.env.sz[1]):
                p=self.policy[(i,j)]
                out = self.env.action_text[p]
                print(f'{out} |',end='')
    def print_v(self, decimal = 1):
        for i in range(self.env.sz[0]):
            print('\n---------------')
            for j in range(self.env.sz[1]):
                out=np.round(self.v[(i,j)],decimal)
                print(f'{out} |',end='')
    def run_episode(self, start, epsilon, first_action = None):
        result = []
        state = self.env.reset(start)
        #dictate first action to iterate q
        if first_action is not None:
            action = first_action
            next_state,reward,done = self.env.step(action)
            result.append((state,action,reward,next_state,done))
            state = next_state
            if done: return(result)
        while True:
            action = self.select_action(state,epsilon)
            next_state,reward,done = self.env.step(action)
            result.append((state,action,reward,next_state,done))
            state = next_state
            if done: break
        return(result)
    def update_policy_q(self):
        for state in self.env.state_space:
            self.policy[state] = np.argmax(self.q[state])
    def mc_predict_v(self,n_episode=10000,first_visit=True):
        for t in range(n_episode):
            traversed = []
            e = self.get_epsilon(t)
            transitions = self.run_episode(self.env.start,e)
            states,actions,rewards,next_states,dones = zip(*transitions)
            for i in range(len(transitions)):
                if first_visit and (states[i] not in traversed):
                    traversed.append(states[i])
                    self.n_v[states[i]]+=1
                    discounts = np.array([self.gamma**j for j in range(len(transitions)+1)])
                    self.v[states[i]]+= sum(rewards[i:]*discounts[:-(1+i)])
        for state in self.env.state_space:
            if state != self.env.goal:
                self.v[state] = self.v[state] / self.n_v[state]
            else:
                self.v[state] = 0
    
    def mc_predict_q(self,n_episode=10000,first_visit=True):
        for t in range(n_episode):
            traversed = []
            e = self.get_epsilon(t)
            transitions = self.run_episode(self.env.start,e)
            states,actions,rewards,next_states,dones = zip(*transitions)
            for i in range(len(transitions)):
                if first_visit and ((states[i],actions[i]) not in traversed):
                    traversed.append((states[i],actions[i]))
                    self.n_q[states[i]][actions[i]]+=1
                    discounts = np.array([self.gamma**j for j in range(len(transitions)+1)])
                    self.q[states[i]][actions[i]]+= sum(rewards[i:]*discounts[:-(1+i)])
                elif not first_visit:
                    self.n_q[states[i]][actions[i]]+=1
                    discounts = np.array([self.gamma**j for j in range(len(transitions)+1)])
                    self.q[states[i]][actions[i]]+= sum(rewards[i:]*discounts[:-(1+i)])

        #print(self.q,self.n_q)
        for state in self.env.state_space:
            for action in range(self.n_action):
                if state != self.env.goal:
                    self.q[state][action] = self.q[state][action] / self.n_q[state][action]
                else:
                    self.q[state][action] = 0
        
    def mc_control_q(self,n_episode=10000,first_visit=True):
        """
        Write the code to perform Monte Carlo Control
        Hint: You just need to do prediction then update the policy
        """
        pass
        
    def mc_control_glie(self,n_episode=10000,first_visit=True,lr=0.):
        """
        Taking hints from the mc_predict_q and mc_control_q methods, write the code to
        perform GLIE Monte Carlo control.
        """
        pass

![RL Framework](img/rl_framework.png)
Source: [Sutton and Barto](https://cdn.preterhuman.net/texts/science_and_technology/artificial_intelligence/Reinforcement%20Learning%20%20An%20Introduction%20-%20Richard%20S.%20Sutton%20,%20Andrew%20G.%20Barto.pdf)

## Solving Reinforcement Learning Problems - Monte Carlo Methods

There are two main approaches in solving reinforcement learning: **model-based** and **model-free** approaches. A model-based approach assumes that we have some or full knowledge of how our environment works whereas a model-free approach relies on our agent to explore the environment without any prior knowledge. 

In this workshop, we will focus on model-free approaches which usually involves two steps: evalulating the state or action value function based on the agent's interactions with the environment also known as **prediction problem** and changing the agent's policy to be closer to an optimal policy also known as **control problem**.

We start with the Monte Carlo Methods aka the trial-and-error-until-you-get-rich-or-broke methods.

![Monte Carlo](img/monte_carlo.jpg)

### Prediction Problem

In [4]:
#stochastic environment
env = Gridworld(wind_p=0.2)

#initial policy
policy = {(0, 0): 3,
          (0, 1): 3,
          (0, 2): 2,
          (1, 0): 3,
          (1, 1): 3,
          (1, 2): 0,
          (2, 0): 3,
          (2, 1): 0,
          (2, 2): 0}

#stochastic agent - epsilon greedy with decays
a = GridworldAgent(env, policy = policy, gamma = 0.9, 
            start_epsilon=0.9,end_epsilon=0.3,epsilon_decay=0.9)

print('Reward Grid')
env.print_reward()
print('\n')
print('Policy: Reach Goal ASAP')
a.print_policy()

Reward Grid

----------
0 |0 |0 |
----------
0 |-5 |5 |
----------
0 |0 |0 |

Policy: Reach Goal ASAP

----------
R |R |D |
----------
R |R |U |
----------
R |U |U |

#### Monte Carlo State Value Prediction

![Monte Carlo State Value Prediction](img/mc_predict_v.png)

In [5]:
a.mc_predict_v()
a.print_v()


---------------
-1.2 |0.8 |2.8 |
---------------
-3.6 |2.0 |0 |
---------------
-3.9 |-3.4 |2.5 |

#### Monte Carlo Action Value Prediction

![Monte Carlo Action Value Prediction](img/mc_predict_q.png)

In [6]:
a.mc_predict_q(first_visit=False)
print(f'\nActions: {env.action_text}')
for i in a.q: print(i,a.q[i])


Actions: ['U' 'L' 'D' 'R']
(2, 0) [-4.27945999 -4.6426242  -4.61743906 -3.84667845]
(1, 0) [-1.80519107 -4.55712767 -4.64543308 -3.69798627]
(1, 1) [-0.6984784  -4.13869325 -3.82360772  3.31915168]
(0, 1) [-0.48133728 -2.19533552 -3.60252915  1.44325715]
(2, 1) [-3.62305429 -4.53351496 -4.16557244  1.06611973]
(0, 2) [ 1.58997974 -0.48889059  3.27806785  1.37156515]
(2, 2) [ 3.31682568 -3.70868514  1.24422577  1.21949238]
(0, 0) [-1.8066222  -1.92334208 -3.46354537 -0.44686229]
(1, 2) [0. 0. 0. 0.]


### Control Problem

In [7]:
#stochastic environment
env = Gridworld(wind_p=0.2)

#initial policy
policy = {(0, 0): 3,
          (0, 1): 3,
          (0, 2): 2,
          (1, 0): 3,
          (1, 1): 3,
          (1, 2): 0,
          (2, 0): 3,
          (2, 1): 0,
          (2, 2): 0}

#stochastic agent - epsilon greedy with decays
a = GridworldAgent(env, policy = policy, gamma = 0.9, 
            start_epsilon=0.9,end_epsilon=0.3,epsilon_decay=0.9)

print('Reward Grid')
env.print_reward()
print('\n')
print('Policy: Reach Goal ASAP')
a.print_policy()

Reward Grid

----------
0 |0 |0 |
----------
0 |-5 |5 |
----------
0 |0 |0 |

Policy: Reach Goal ASAP

----------
R |R |D |
----------
R |R |U |
----------
R |U |U |

### All-visit Monte Carlo

**Coding Assignment** Implement `mc_control_q` function of `agent.py` using either all-visit or first-visit Monte Carlo.

In [8]:
#reset
a.policy = policy
a.q = defaultdict(lambda: np.zeros(a.n_action))
a.n_q = defaultdict(lambda: np.zeros(a.n_action))

a.mc_control_q(n_episode = 1000,first_visit=False)
a.print_policy()
print(f'\nActions: {env.action_text}')
for i in a.q: print(i,a.q[i])


----------
R |R |D |
----------
U |R |U |
----------
R |R |U |
Actions: ['U' 'L' 'D' 'R']
(2, 0) [-4.42542673 -4.47945424 -4.96856101 -3.83328205]
(2, 1) [-3.71362126 -4.60951413 -3.74927154  0.60503252]
(2, 2) [ 3.30503944 -4.48875665  1.9600145  -0.5514421 ]
(1, 1) [-0.3975418  -4.66350337 -3.90694783  3.30577735]
(1, 0) [-2.47231007 -6.12862609 -4.24574936 -3.90817076]
(0, 1) [-1.58418182e-03 -1.47717932e+00 -5.39762380e+00  1.57492868e+00]
(0, 2) [ 1.08329745 -1.38704065  3.41271362  0.95096   ]
(0, 0) [-2.66507277 -3.9394442  -6.079117   -1.28748741]
(1, 2) [0. 0. 0. 0.]


#### First-visit Monte Carlo

In [9]:
#reset
a.policy = policy
a.q = defaultdict(lambda: np.zeros(a.n_action))
a.n_q = defaultdict(lambda: np.zeros(a.n_action))

a.mc_control_q(n_episode = 1000,first_visit=True)
a.print_policy()
print(f'\nActions: {env.action_text}')
for i in a.q: print(i,a.q[i])


----------
R |R |D |
----------
D |R |U |
----------
R |R |U |
Actions: ['U' 'L' 'D' 'R']
(2, 0) [-2.43309971 -1.74925805 -1.53901673 -0.55055766]
(2, 1) [-3.03968978 -1.79147033 -0.32667539  1.48590312]
(1, 1) [-0.37956211 -1.95397097 -0.03600181  2.9814032 ]
(1, 0) [-1.59538693 -2.50784238 -0.898435   -4.03042833]
(2, 2) [ 3.37561369 -1.18148985  1.25791886  1.75502656]
(0, 0) [-2.48601718 -1.54522598 -3.20786409 -0.27791786]
(0, 2) [ 1.6016075  -0.0454231   3.46291667  1.76135   ]
(0, 1) [-0.27251285 -2.15225689 -3.01662     1.5304814 ]
(1, 2) [0. 0. 0. 0.]


#### Greedy within The Limit of Exploration 

![Greedy within The Limit of Exploration](img/mc_control_glie.png)

**Coding Assignment** Implement `mc_control_glie` function of `agent.py`

In [10]:
#reset
a.policy = policy
a.q = defaultdict(lambda: np.zeros(a.n_action))
a.n_q = defaultdict(lambda: np.zeros(a.n_action))

a.mc_control_glie(n_episode = 1000)
a.print_policy()
print(f'\nActions: {env.action_text}')
for i in a.q: print(i,a.q[i])


----------
R |R |D |
----------
D |R |U |
----------
R |R |U |
Actions: ['U' 'L' 'D' 'R']
(2, 0) [-3.3134489  -2.67478023 -2.1137735  -0.49593229]
(0, 0) [ -3.86583878 -10.08194964  -5.95308958  -0.82373368]
(0, 1) [-2.36453588 -1.89078656 -4.27948146  1.39288345]
(0, 2) [ 1.464686   -0.79093121  3.39272518  1.82825   ]
(1, 0) [-3.01087594 -2.94596132 -2.12747573 -6.28092154]
(1, 1) [-0.47948514 -3.53229022 -2.19359228  3.14234656]
(1, 2) [0. 0. 0. 0.]
(2, 1) [-3.06808843 -1.97433774 -0.18890794  1.5132909 ]
(2, 2) [ 3.30062164 -0.79716753  1.69313086  1.44001816]


#### GLIE with Constant Learning Rate

![GLIE with constant learning rate](img/mc_control_glie_constant.png)

In [11]:
#reset
a.policy = policy
a.q = defaultdict(lambda: np.zeros(a.n_action))
a.n_q = defaultdict(lambda: np.zeros(a.n_action))

a.mc_control_glie(n_episode = 1000, lr=0.1)
a.print_policy()
print(f'\nActions: {env.action_text}')
for i in a.q: print(i,a.q[i])


----------
R |R |D |
----------
D |R |U |
----------
R |R |U |
Actions: ['U' 'L' 'D' 'R']
(2, 0) [-2.81750386 -2.74814452 -2.49249333 -1.06695963]
(0, 0) [-2.60372804 -2.60435885 -2.71952252 -0.90359939]
(0, 1) [-2.20346185 -1.64821088 -1.77704951  0.88879714]
(0, 2) [ 0.66803909 -0.36253117  3.23068326  0.21412206]
(1, 0) [-3.33687023 -3.23160784 -1.68694516 -5.1361141 ]
(1, 1) [-1.29011076 -2.06865674  0.23615334  3.60073613]
(1, 2) [0. 0. 0. 0.]
(2, 1) [-2.92641498 -1.39443037 -1.57252958  1.54730252]
(2, 2) [ 3.14731041 -1.10108852  0.84096201  1.27417214]
